## Data preprocessing

In [151]:
# Filter the warnings
import warnings

warnings.filterwarnings('ignore')

**Importing the libraries**

In [152]:
# Miscellaneous libraries
import numpy as np
import pandas as pd

# For pretty display of pandas dataframes in the notebook
from IPython.display import display, HTML

**Reading the data**

Read file "20years.csv" from ./DATA/ folder inside the working directory. Take only the columns related with temperatures and parse the dates.

In [153]:
# Relative path to data file
data_path = r'./DATA/20years.csv'

# Read the file
data = pd.read_csv(
    data_path, usecols = ['DATE', 'TMAX', 'TMIN', 'TAVG', 'PRCP', 'SNOW'], parse_dates = ['DATE']
)

**Delete the 29 of February from the data, to avoid possible problems with pandas DateTimes**

In [154]:
# Locate the 29th of february indices
february29 = list(data[(data.DATE.dt.month == 2) & (data.DATE.dt.day == 29)].index)

# Delete those indices from the data
data = data.drop(february29).reset_index(drop = True)

**Missing values**

In [155]:
pd.concat(
    [
        data.isnull().sum(),
        (data.isnull().sum() / data.isnull().count()*100)
    ], 
    axis = 1, keys = ['Total', 'Percent']
)

,Total,Percent
DATE,0,0.000000
PRCP,0,0.000000
SNOW,0,0.000000
TAVG,2894,37.756034
TMAX,0,0.000000
TMIN,0,0.000000


In [156]:
print(f'There are a total of {data.TAVG.isna().sum()} missing values for the average temperatures.')

There are a total of 2894 missing values for the average temperatures.


Imput the missing values with the mean between the maximum and the minimum of that same day:

In [157]:
data['TAVG'] = data.apply(
    lambda row: (row['TMIN'] + row['TMAX'])/2 if np.isnan(row['TAVG']) else row['TAVG'], axis = 1
)

In [158]:
print(f'There are a total of {data.TAVG.isna().sum()} missing values for the average temperatures.')

There are a total of 0 missing values for the average temperatures.


**Computing the historical values**

Group the values by day of the year and compute the means and the max-min of every day:

In [159]:
historical = data.groupby(
    [data.DATE.dt.month, data.DATE.dt.day]
).agg(
    {
        'PRCP' : 'mean',
        'SNOW' : 'mean',
        'TMAX' : ['mean', 'max'],
        'TMIN' : ['mean', 'min'],
        'TAVG' : 'mean'
    }
)

Notice that we have problems with multilevel indices and multilevel column indices. We will have to fix that.

In [160]:
historical.head()

PRCP      SNOW      TMAX            TMIN            TAVG
               mean      mean      mean   max      mean   min      mean
DATE DATE                                                              
1    1     3.076190  0.000000  5.104762  16.1 -2.161905 -13.8  1.600000
     2     1.295238  3.047619  4.666667  12.2 -2.866667 -12.2  1.028571
     3     8.228571  6.904762  4.600000  15.6 -2.057143 -12.7  1.285714
     4     2.247619  9.666667  5.519048  15.6 -2.390476 -16.0  1.723810
     5     1.604762  3.285714  4.919048  13.3 -2.271429 -12.7  1.473810

In [161]:
# Flatten the multiIndex in the column space
historical.columns = ['_'.join(col).strip() for col in historical.columns.values]

In [162]:
# Flatten the multiIndex in the index space

# Flatten the index into tuples
historical.index = historical.index.to_flat_index()
# Convert tuple objects to strings separated by '-'
historical.index = [str(indx1) + '-' + str(indx2) for indx1, indx2 in historical.index.values]
# Convert strings 'm-d' to datetime object from 2010, for joining purposes later on
historical.index = pd.to_datetime(historical.index, format = '%m-%d')
# Set index name
historical.index.name = 'DATE'

# Finally, reset the index in order to have Date as a column
historical = historical.reset_index()

Append a column of 0's, and the total maximum and the total minimum (for the last 20 years).

In [163]:
# Append the all-zeroes column
historical['zero'] = [0] * 365

# Append the all-time-maximum temperature column
maximum = historical.TMAX_max.max()
maxvect = [maximum] * 365
historical['total_max'] = maxvect

# Append the all-time-minimum temperature column
minimum = historical.TMIN_min.min()
minvect = [minimum] * 365
historical['total_min'] = minvect

Display the historical dataframe:

In [164]:
display(historical)

,DATE,PRCP_mean,SNOW_mean,TMAX_mean,TMAX_max,TMIN_mean,TMIN_min,TAVG_mean,zero,total_max,total_min
0,1900-01-01,3.076190,0.000000,5.104762,16.1,-2.161905,-13.8,1.600000,0,39.4,-17.2
1,1900-01-02,1.295238,3.047619,4.666667,12.2,-2.866667,-12.2,1.028571,0,39.4,-17.2
2,1900-01-03,8.228571,6.904762,4.600000,15.6,-2.057143,-12.7,1.285714,0,39.4,-17.2
3,1900-01-04,2.247619,9.666667,5.519048,15.6,-2.390476,-16.0,1.723810,0,39.4,-17.2
4,1900-01-05,1.604762,3.285714,4.919048,13.3,-2.271429,-12.7,1.473810,0,39.4,-17.2
...,...,...,...,...,...,...,...,...,...,...,...
360,1900-12-27,2.542857,7.142857,6.366667,17.2,-0.095238,-8.2,3.166667,0,39.4,-17.2
361,1900-12-28,1.323810,0.285714,6.480952,17.8,-1.023810,-11.6,2.973810,0,39.4,-17.2
362,1900-12-29,4.609524,0.476190,6.000000,12.2,-0.671429,-11.0,2.869048,0,39.4,-17.2
363,1900-12-30,1.514286,13.190476,5.509524,12.2,-1.519048,-8.2,2.207143,0,39.4,-17.2


**Cumulative precipitations**

We compute the cumulative precipitations for every month of every year of the data:

In [165]:
for year in range(1998, 2019):
    # Sub-dataframe with that specific year
    data_year = data[data.DATE.dt.year == year].reset_index(drop = True)
    
    # Vector with the new ppt for this year
    new_ppt = [np.nan] * 365
    old_ppt = [obs[1] for obs in data_year.values]
    
    # For every day of the year
    new_ppt[0] = old_ppt[0]
    for i in range(1, 365):
        if data_year.iloc[i].DATE.date().month == data_year.iloc[i-1].DATE.date().month:
            new_ppt[i] = old_ppt[i] + new_ppt[i-1]
        else:
            new_ppt[i] = old_ppt[i]
            
    data.PRCP[data.DATE.dt.year == year] = new_ppt 

Display the data:

In [166]:
display(data)

,DATE,PRCP,SNOW,TAVG,TMAX,TMIN
0,1998-01-01,0.0,0.0,-5.00,-0.6,-9.4
1,1998-01-02,0.0,0.0,3.05,7.8,-1.7
2,1998-01-03,0.0,0.0,8.60,12.8,4.4
3,1998-01-04,0.3,0.0,11.15,15.6,6.7
4,1998-01-05,0.3,0.0,7.20,9.4,5.0
...,...,...,...,...,...,...
7660,2018-12-27,115.6,0.0,4.40,6.7,2.8
7661,2018-12-28,140.5,0.0,8.00,12.8,6.1
7662,2018-12-29,140.5,0.0,10.20,12.2,3.9
7663,2018-12-30,140.5,0.0,3.00,3.9,0.0


**Computing the max-min all-time records for every year**

In [167]:
records = []

for year, year_data in data.groupby(data.DATE.dt.year):
    year_data = year_data.reset_index(drop = True)
    year_maxRecords = list(year_data.TMAX >= historical.TMAX_max)
    year_minRecords = list(year_data.TMIN <= historical.TMIN_min)
    
    # Apend dictionaries to the list 'records' to avoid problems with dataframes
    # Each dictionary represents one row
    for maximum in year_data.iloc[year_maxRecords].values:
        new_record = {
            'DATE' : maximum[0],
            'temp' : maximum[4],
            'kind' : 'Max'
        }
        records.append(new_record)
    for minimum in year_data.iloc[year_minRecords].values:
        new_record = {
            'DATE' : minimum[0],
            'temp' : minimum[5],
            'kind' : 'Min'
        }
        records.append(new_record)

# Convert the list of rows to a pandas DataFrame
records = pd.DataFrame(records)

**Save the files to the ./DATA/ directory**

In [168]:
historical.to_csv(r'./DATA/historical.csv', index = False)
data.to_csv(r'./DATA/allYears.csv', index = False)
records.to_csv(r'./DATA/records.csv', index = False)